In [1]:
import plotly.express as px
import plotly.io as pio

#pio.renderers.default = "iframe_connected"
pio.renderers.default = "vscode"


In [2]:
from pathlib import Path
import sys

# Go two levels up: from notebooks/ -> parenthood_europe/ -> CodingProjects/
project_root = Path().resolve().parents[0]
sys.path.insert(0, str(project_root))
sys.path.append (str(project_root)+"/libs")


In [3]:
list(Path().resolve().parents)


[PosixPath('/Users/Paula_1/CodingProjects/parenthood/parenthood_europe'),
 PosixPath('/Users/Paula_1/CodingProjects/parenthood'),
 PosixPath('/Users/Paula_1/CodingProjects'),
 PosixPath('/Users/Paula_1'),
 PosixPath('/Users'),
 PosixPath('/')]

In [4]:
(str(project_root)+"/libs")


'/Users/Paula_1/CodingProjects/parenthood/parenthood_europe/libs'

In [5]:
from scripts.parse_survey_data import load_survey_data
from libs.survey_analysis import NumericQuestion, SingleChoiceQuestion, MultipleChoiceQuestion, MatrixQuestion
from libs.transform_time import months_to_weeks, unified_time_to_weeks

In [6]:
from pathlib import Path
print("Current working directory:", Path().resolve())


Current working directory: /Users/Paula_1/CodingProjects/parenthood/parenthood_europe/notebooks


In [7]:
df, df_raw = load_survey_data(file_path = "../data/parenthood_test_QNumeric.xlsx")
print(df.loc[1, "DE1"])


In what year were you born?


In [8]:
df, df_raw = load_survey_data(file_path = "../data/parenthood_test_QNumeric.xlsx")

q1 = NumericQuestion("DE1", df, df_raw)
fig1 = q1.distribution(display=False)
fig1.show()

In [9]:
print([col for col in df.columns if col.startswith("DE10")])


['DE10_1', 'DE10_3', 'DE10_4', 'DE10_5', 'DE10_6', 'DE10_7', 'DE10_9']


In [10]:
numeric_question_ids = ["DE1", "DE10", "DE11", "DE22"]
for qid in numeric_question_ids:
    q = NumericQuestion(qid, df, df_raw)
    fig = q.distribution(display=False)
    if fig is not None:
        fig.show()


In [11]:
df, df_raw = load_survey_data(file_path = "../data/parenthood_test_QSingle.xlsx")

single_choice_question_ids = ["DE2", "DE4", "DE20"]
figs = []
for qid in single_choice_question_ids:
    q = SingleChoiceQuestion(qid, df, df_raw)
    fig = q.distribution(display=False)
    figs.append(fig)

for fig in figs:
    if fig is not None:
        fig.show()

In [12]:
print("Column DE2 type:", df["DE2"].dtype)
print("Unique values in DE2:", df["DE2"].unique())


Column DE2 type: object
Unique values in DE2: ['DE2' 'What is your gender identity?' 1 2 nan 3 4]


In [13]:
value = df["DE2"].iloc[3]  # 3 means the 4th row (0-based indexing)
print("Value at row 3, col 'DE2':", value)
print("Type of that value:", type(value))


Value at row 3, col 'DE2': 2
Type of that value: <class 'int'>


In [14]:
df, df_raw = load_survey_data(file_path = "../data/parenthood_test_QMultiple.xlsx")
print(df.loc[1, "DE3"])

What are your ethnic origins or ancestry? Please, select ALL the geographic areas from which your family's ancestors first originated: - Selected Choice


In [15]:
value = df["DE3"].iloc[3]  
print("Value at row 3, col 'DE3':", value)
print("Type of that value:", type(value))


Value at row 3, col 'DE3': 1
Type of that value: <class 'str'>


In [16]:
# MultipleChoice Question

df, df_raw = load_survey_data(file_path="../data/parenthood_test_QMultiple.xlsx")

multiple_choice_question_ids = ["DE3", "DE7", "DE9"]

for qid in multiple_choice_question_ids:
    q = MultipleChoiceQuestion(qid, df, df_raw)
    fig = q.distribution(display=False)
    if fig is not None:
        fig.show()


In [6]:
# MatrixChoice Question

df, df_raw = load_survey_data(file_path="../data/parenthood_test_QMatrix.xlsx")

multiple_choice_question_ids = ["DE14", "DE15", "DE16"]

for qid in multiple_choice_question_ids:
    q = MatrixQuestion(qid, df, df_raw)
    fig = q.distribution(display=False)
    if fig is not None:
        fig.show()
